<a href="https://colab.research.google.com/github/jerry26432341/myTheses/blob/main/0705meeting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -O chineseStopWords.txt https://raw.githubusercontent.com/tomlinNTUB/Machine-Learning-Basics/main/Chinese%20Stopwords.txt

--2023-07-22 03:45:05--  https://raw.githubusercontent.com/tomlinNTUB/Machine-Learning-Basics/main/Chinese%20Stopwords.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-07-22 03:45:05 ERROR 404: Not Found.



In [ ]:
import pandas as pd
import numpy as np
import jieba
import re
import gensim
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, vstack
from sklearn.svm import LinearSVC
from joblib import dump
from sklearn.model_selection import train_test_split
from google.colab import drive

# 掛載Google Drive
drive.mount('/content/drive')

# 读取数据
df = pd.read_csv('/content/drive/MyDrive/shopping.csv')

Mounted at /content/drive


In [ ]:
# 检查空值
print(df.isnull().sum())

# 删除包含空值的行
df = df.dropna()


cat       0
label     0
review    1
dtype: int64


In [ ]:
# 定义文本清洗函数
def clean_text(text):
    # 去除特殊字符和标点符号
    text = re.sub(r"[^\w\s]", "", text)
    # 转换为小写
    text = text.lower()
    return text

# 清洗文本数据
df['cleaned_review'] = df['review'].apply(clean_text)


In [ ]:
# 定义分词函数
def tokenize_text(text):
    tokens = jieba.cut(text)
    return " ".join(tokens)

# 分词处理
df['tokenized_review'] = df['cleaned_review'].apply(tokenize_text)


Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.622 seconds.
DEBUG:jieba:Loading model cost 0.622 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [ ]:
# 定义停用词列表
stopwords = []

# 加载停用词列表
with open('chineseStopWords.txt', 'r', encoding='utf-8') as f:
    stopwords = [line.strip() for line in f.readlines()]

# 定义停用词过滤函数
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return " ".join(filtered_words)

# 停用词过滤
df['filtered_review'] = df['tokenized_review'].apply(remove_stopwords)

# 创建一个新的数据框存储预测结果
df_predicted = pd.DataFrame({'review': df['review'], 'predicted_category': None})


In [ ]:
import scipy as sp
from scipy.sparse import hstack

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# 使用TF-IDF进行特征提取
tfidf_vectorizer = TfidfVectorizer()
tfidf_features = tfidf_vectorizer.fit_transform(df['filtered_review'])

# 使用Doc2Vec进行特征提取
tagged_documents = [TaggedDocument(words=jieba.lcut(text), tags=[i]) for i, text in enumerate(df['filtered_review'])]
doc2vec_model = Doc2Vec(tagged_documents, vector_size=100, window=5, min_count=5, epochs=10)
doc2vec_features = np.array([doc2vec_model.infer_vector(jieba.lcut(text)) for text in df['filtered_review']])

# 將doc2vec_features轉換為稀疏矩陣
doc2vec_features_sparse = sp.sparse.csr_matrix(doc2vec_features)

# 合併TF-IDF和Doc2Vec特徵
feature_matrix = sp.sparse.hstack((tfidf_features, doc2vec_features_sparse))

# 打印特征矩阵的维度
print("特征矩阵维度:", feature_matrix.shape)


特征矩阵维度: (62773, 68232)


In [ ]:
# 计算每个类别的80%数据数量
category_counts = df['cat'].value_counts()
test_size_percent = 0.8
category_test_sizes = (category_counts * test_size_percent).astype(int)
print(category_test_sizes)

平板     8000
水果     8000
洗发水    8000
衣服     8000
酒店     8000
计算机    3193
书籍     3080
手机     1858
蒙牛     1626
热水器     459
Name: cat, dtype: int64


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.sparse import vstack

# 创建空的训练集和测试集索引
X_train_indices = []
X_test_indices = []

# 遍历每个类别
for category in category_test_sizes.index:
    category_indices = df[df['cat'] == category].index
    category_size = len(category_indices)

    # 计算用于训练的样本数量
    train_size = int(category_size * train_ratio)

    # 随机选择训练集的索引
    train_indices = np.random.choice(category_indices, size=train_size, replace=False)

    # 将剩余的索引作为测试集的索引
    test_indices = list(set(category_indices) - set(train_indices))

   # 确保测试集索引在有效范围内
    test_indices = [idx for idx in test_indices if idx < feature_matrix.shape[0]]

# 将训练集和测试集的索引添加到最终的索引列表中
X_train_indices.extend(train_indices)
X_test_indices.extend(test_indices)


# 将索引转换为特征矩阵的稀疏矩阵索引
X_train_indices_sparse = np.array(X_train_indices)
X_test_indices_sparse = np.array(X_test_indices)

# 打印特征矩阵的维度
print("特征矩阵维度:", feature_matrix.shape)
print("训练集索引范围:", X_train_indices_sparse.min(), "-", X_train_indices_sparse.max())
print("测试集索引范围:", X_test_indices_sparse.min(), "-", X_test_indices_sparse.max())

# 将索引转换为稀疏矩阵
X_train_final = vstack([feature_matrix[i] for i in X_train_indices_sparse])
X_test_final = vstack([feature_matrix[i] for i in X_test_indices_sparse])

# 打印最终的训练集和测试集维度
print("最终训练集维度:", X_train_final.shape)
print("最终测试集维度:", X_test_final.shape)


NameError: ignored

In [ ]:
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

# 创建LinearSVC模型
model = LinearSVC()

# 在训练集上训练模型
model.fit(X_train_final, df.loc[X_train_indices_sparse, 'cat'])

# 在测试集上进行预测
predictions = model.predict(X_test_final)

# 打印分类报告
print(classification_report(df.loc[X_test_indices_sparse, 'cat'], predictions))


In [ ]:
import pandas as pd
from google.colab import files

# 重置X_test的索引并创建一个新的DataFrame
X_test_df = pd.DataFrame(X_test_final.toarray())
X_test_df.reset_index(drop=True, inplace=True)

# 确保X_test_df和predictions的长度一致
X_test_df = X_test_df[:len(predictions)]

# 创建一个新的数据框存储预测结果
df_predicted = pd.DataFrame({'review': df.loc[X_test_indices_sparse, 'review'],
                             'predicted_category': predictions})

# 获取预测概率
probabilities = model.decision_function(X_test_final)

# 获取前三个最高概率的类别和概率值
top_three_indices = np.argsort(probabilities, axis=1)[:, -3:][:, ::-1]
top_three_categories = model.classes_[top_three_indices]
top_three_probabilities = probabilities[np.arange(len(predictions))[:, np.newaxis], top_three_indices]

# 添加类别和概率列
for i in range(3):
    df_predicted[f'top_{i+1}_category'] = top_three_categories[:, i]
    df_predicted[f'top_{i+1}_probability'] = top_three_probabilities[:, i]

# 输出预测结果
print(df_predicted)

# 将预测结果保存为 Excel 文件
df_predicted.to_excel('predictions_with_probabilities.xlsx', index=False)
# files.download('predictions_with_probabilities.xlsx')


In [ ]:
# 读取原始数据
df_original = pd.read_csv('/content/drive/MyDrive/shopping.csv')

# 匹配预测结果和原始数据
df_predicted['true_category'] = df_original['cat'].values[df_predicted.index]

# 添加一列用于标识预测的正确性
df_predicted['prediction_correct'] = (df_predicted['predicted_category'] == df_predicted['true_category'])

# 进一步分析和区分预测的正确和错误
# 正确的预测
correct_predictions = df_predicted[df_predicted['prediction_correct'] == True]

# 错误的预测
incorrect_predictions = df_predicted[df_predicted['prediction_correct'] == False]

# 将正确的预测保存为 Excel 文件
correct_predictions.to_excel('correct_predictions.xlsx', index=False)
files.download('correct_predictions.xlsx')

# 将错误的预测保存为 Excel 文件
incorrect_predictions.to_excel('incorrect_predictions.xlsx', index=False)
files.download('incorrect_predictions.xlsx')


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# 读取真实标签和预测结果
y_true = df_predicted['true_category']
y_pred = df_predicted['predicted_category']

# 计算准确率
accuracy = accuracy_score(y_true, y_pred)

# 计算召回率
recall = recall_score(y_true, y_pred, average='weighted')

# 计算精确率
precision = precision_score(y_true, y_pred, average='weighted')

# 计算F1分数
f1 = f1_score(y_true, y_pred, average='weighted')

# 输出指标结果
print("准确率:", accuracy)
print("召回率:", recall)
print("精确率:", precision)
print("F1分数:", f1)
